In [1]:
import sys
import tensorflow.keras
import pandas as pd
import sklearn as sk
import tensorflow as tf
import platform

print(f"Python Platform: {platform.platform()}")
print(f"Tensor Flow Version: {tf.__version__}")
print(f"Keras Version: {tensorflow.keras.__version__}")
print()
print(f"Python {sys.version}")
print(f"Pandas {pd.__version__}")
print(f"Scikit-Learn {sk.__version__}")
gpu = len(tf.config.list_physical_devices('GPU'))>0
print("GPU is", "available" if gpu else "NOT AVAILABLE")

Python Platform: macOS-13.2.1-arm64-arm-64bit
Tensor Flow Version: 2.12.0
Keras Version: 2.12.0

Python 3.10.10 | packaged by conda-forge | (main, Mar 24 2023, 20:12:31) [Clang 14.0.6 ]
Pandas 2.1.1
Scikit-Learn 1.2.2
GPU is available


In [2]:
# IMPORTING STATEMENTS
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#import mutual_information_regression as mri
#import helper_functions_fs as hf

np.random.seed(7)

In [3]:
from sklearn.preprocessing import StandardScaler

# Import the Data

In [4]:
# training data
data = pd.read_csv('../../0_data/train.csv', index_col=0).reset_index(drop=True)

In [5]:
# defining response variable and input features
y = data.pop('InterestRate')
X = data.copy()

In [6]:
data_test = pd.read_csv('TEST_DATA.csv', index_col=0).reset_index(drop=True)     # modificato
data_test.head()

,InterestRate,MinorityRatio,CensusTractMedFamIncome,MonthlyIncome,HUDMedIncome,LTV,PaymentCount,Age1,Age2,HousingExpense,...,State_31,State_39,State_42,State_48,State_55,State_infrequent_sklearn,Minority_Min,Minority_Missing,Minority_White,Borrower2Dummy
0,4.500,16.936,154125,14425,119600,72.0,360,30,30,35.82,...,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,1
1,5.000,22.466,156053,15803,79200,90.0,360,50,50,18.72,...,0.0,0.0,0.0,0.0,0.0,1.0,1,0,0,1
2,2.625,2.467,72944,9724,76600,80.0,180,33,0,17.10,...,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,0
3,3.625,14.460,109038,7044,99300,90.0,360,32,33,21.42,...,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,1
4,3.750,4.777,143165,16373,109200,72.0,360,49,49,13.86,...,0.0,0.0,0.0,0.0,0.0,1.0,1,0,0,1


In [7]:
y_test = data_test.pop('InterestRate')
X_test = data_test.copy()

# Scaling

In [8]:
numerical = pd.read_csv('../../0_data/numerical.csv', index_col=0)
numerical.iloc[:,0]

0                InterestRate
1               MinorityRatio
2     CensusTractMedFamIncome
3               MonthlyIncome
4                HUDMedIncome
5                         LTV
6                PaymentCount
7                        Age1
8                        Age2
9              HousingExpense
10                DebtExpense
11                        PMI
12               CreditScore1
13               CreditScore2
14                   NoteDate
15                 NoteAmount
Name: 0, dtype: object

In [9]:
# separate numerical and dummies

X_num = pd.concat([X.pop(col) for col in list(numerical.iloc[1:,0])], axis=1)
X['BorrowerCount'] = X['BorrowerCount']-1

X_test_num = pd.concat([X_test.pop(col) for col in list(numerical.iloc[1:,0])], axis=1)
X_test['BorrowerCount'] = X_test['BorrowerCount']-1

In [10]:
# scale numerical vars 
ss = StandardScaler()
X_train_num = ss.fit_transform(X_num)
X_test_num = ss.transform(X_test_num)
X_num = pd.DataFrame(X_train_num, columns=list(numerical.iloc[1:,0]))
X_test_num = pd.DataFrame(X_test_num, columns=list(numerical.iloc[1:,0]))
X_num.describe()

/Users/fra/miniconda3/envs/tensorflow/lib/python3.10/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/Users/fra/miniconda3/envs/tensorflow/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/fra/miniconda3/envs/tensorflow/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/fra/miniconda3/envs/tensorflow/lib/python3.10/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated

,MinorityRatio,CensusTractMedFamIncome,MonthlyIncome,HUDMedIncome,LTV,PaymentCount,Age1,Age2,HousingExpense,DebtExpense,PMI,CreditScore1,CreditScore2,NoteDate,NoteAmount
count,1.890790e+05,1.890790e+05,1.890790e+05,1.890790e+05,1.890790e+05,1.890790e+05,1.890790e+05,1.890790e+05,1.890790e+05,1.890790e+05,1.890790e+05,1.890790e+05,1.890790e+05,1.890790e+05,1.890790e+05
mean,-2.335544e-16,4.329118e-17,6.898592e-17,-1.819958e-16,6.964825e-16,2.689164e-16,-2.288664e-16,4.235170e-17,1.323913e-16,6.650381e-16,3.112493e-17,2.442832e-16,2.375942e-16,2.158467e-14,-4.906427e-17
std,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00
min,-9.829129e-01,-2.968603e+00,-1.541693e+00,-3.136269e+00,-8.617748e-01,-3.667909e+00,-2.003581e+00,-1.101219e+00,-2.410365e+00,-3.148844e+00,-4.158231e-01,-4.275040e+00,-2.317892e+00,-7.010553e+00,-1.761821e+00
25%,-6.899229e-01,-6.907823e-01,-6.998868e-01,-6.839963e-01,-8.617748e-01,-2.778056e-01,-8.066413e-01,-1.101219e+00,-7.202951e-01,-7.560665e-01,-4.158231e-01,-5.143845e-01,-5.737875e-01,-1.000778e+00,-7.500204e-01
50%,-3.806037e-01,-2.074440e-01,-2.267679e-01,-6.296614e-02,-9.168695e-02,5.697203e-01,-1.333627e-01,1.920703e-01,-1.531912e-01,8.581993e-03,-4.158231e-01,7.391671e-01,-5.737875e-01,2.011766e-01,-2.017936e-01
75%,2.836096e-01,5.193869e-01,4.021168e-01,5.102925e-01,1.650090e-01,5.697203e-01,6.895334e-01,7.761363e-01,5.862923e-01,7.836339e-01,-4.158231e-01,7.391671e-01,1.170317e+00,1.403131e+00,5.519377e-01
max,3.195775e+00,4.143399e+00,3.701777e+00,3.949844e+00,5.170580e+00,5.697203e-01,3.981118e+00,2.862086e+00,4.372398e+00,5.361121e+00,7.908332e+00,7.391671e-01,1.170317e+00,1.403131e+00,4.433948e+00


In [11]:
# merge back together numerical and dummies
X = pd.concat([X, X_num], axis=1, ignore_index=False)
X_test = pd.concat([X_test, X_test_num], axis=1,ignore_index=False)

In [12]:
X_test

,BorrowerCount,FirstTime,SelfEmployed,Bank_Boston,Bank_Chicago,Bank_Cincinnati,Bank_Dallas,Bank_Des Moines,Bank_Indianapolis,Bank_Pittsburgh,...,PaymentCount,Age1,Age2,HousingExpense,DebtExpense,PMI,CreditScore1,CreditScore2,NoteDate,NoteAmount
0,1,1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.569720,-1.105876,0.150351,2.064010,1.740745,-0.415823,-0.514385,-0.573787,-1.000778,3.998591
1,1,0,1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.569720,0.390298,0.984731,-0.071998,0.784674,2.252175,-3.021488,-1.009814,-1.000778,1.171192
2,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,-1.972857,-0.881450,-1.101219,-0.274357,0.940725,-0.415823,-0.514385,1.170317,1.403131,-0.035713
3,1,1,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.569720,-0.956259,0.275508,0.265267,-0.623944,2.252175,-0.514385,-0.573787,-1.000778,-0.040550
4,1,0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.569720,0.315490,0.943012,-0.679074,-0.677001,-0.415823,-0.514385,-1.009814,-1.000778,0.854349
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2245,1,0,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.569720,-0.657024,0.400665,0.587541,-0.235897,-0.415823,0.739167,-0.573787,-1.000778,-0.298539
2246,0,0,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.569720,0.988768,-1.101219,-0.235634,0.448645,-0.415823,-3.021488,1.170317,1.403131,-1.012847
2247,0,1,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.569720,0.016255,-1.101219,0.540075,1.772995,-0.415823,-1.767936,1.170317,-2.202733,0.322247
2248,1,1,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.569720,-1.255494,0.192070,1.198365,0.087648,2.785775,0.739167,-0.573787,0.201177,0.519770


In [ ]:
X

In [13]:
# dependent variables
ss_y = StandardScaler()
y = ss_y.fit_transform(np.array(y).reshape(-1, 1))
y_test = ss_y.transform(np.array(y_test).reshape(-1, 1))
pd.Series(y.flatten()).describe()

count    1.890790e+05
mean    -1.981173e-16
std      1.000003e+00
min     -2.684678e+00
25%     -7.623710e-01
50%     -1.856790e-01
75%      5.832437e-01
max      4.427857e+00
dtype: float64

In [14]:
y_test

array([[ 1.73662773],
       [ 2.50555042],
       [-1.14683236],
       ...,
       [ 3.08224244],
       [ 0.19878235],
       [ 2.31331975]])

# Scoring Methods


In [15]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error


def mean_residual(y_true, prediction):
    '''Compute the mean residual metric'''
    residuals = (y_true - prediction)
    MR = residuals.mean()
    return MR


def compute_scores(x, y_true, y_pred):
    '''Compute the following metrics for the given model: MSE, MAE, MR, r2'''
    y_true = y_true.reshape(y_pred.shape)

    MSE = mean_squared_error(y_true, y_pred)
    print(f"MSE: {MSE:.4f}")
    MAE = mean_absolute_error(y_true, y_pred)
    print(f"MAE: {MAE:.4f}")
    MR = mean_residual(y_true, y_pred)
    print(f"MR: {MR:.4f}")
    R2 = r2_score(y_true, y_pred)
    adj_R2 = 1- ((1-R2) * (x.shape[0]-1)/(x.shape[0]-x.shape[1]-1))
    print(f"adj-R2: {adj_R2:.4f}")
    return [MSE, MAE, MR, adj_R2]


def save_scores(model, scores):
    '''Store scores for the given model into a dedicated dataframe for a final comparison across models'''
    MSE, MAE, MR, r2 = scores
    #pd.Series(scores).to_csv('scores/'+model)
    model_names.append(model)
    MSEs.append(MSE)
    MAEs.append(MAE)
    MRs.append(MR)
    r2s.append(r2)
    

def display_scores():
    scores_df = pd.DataFrame([model_names, MSEs, MAEs, MRs, r2s], 
                 index=['method', 'MSE', 'MAE', 'MR', 'adj R-squared']).T
    return scores_df

In [16]:
model_names, MSEs, MAEs, MRs, r2s = [],[],[],[],[]

In [17]:
def export_pred(y_pred_std, model, scaler=ss_y):
    '''Reverts model predictions to the original scale of the input data and then saves it to a csv file'''

    y_pred = scaler.inverse_transform(np.array(y_pred_std, ndmin=2)).flatten()
    np.savetxt('predictions/'+str(model)+'_pred.csv', y_pred, delimiter=",")

# MODELS

# 1. Linear Regression

In [18]:
import statsmodels.api as sm

In [19]:
#add constant to predictor variables
x = sm.add_constant(X)
#fit linear regression model
lr = sm.OLS(y, x).fit()

#view model summary
print(lr.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.705
Model:                            OLS   Adj. R-squared:                  0.705
Method:                 Least Squares   F-statistic:                     7061.
Date:                Wed, 04 Oct 2023   Prob (F-statistic):               0.00
Time:                        00:17:27   Log-Likelihood:            -1.5285e+05
No. Observations:              189079   AIC:                         3.058e+05
Df Residuals:                  189014   BIC:                         3.065e+05
Df Model:                          64                                         
Covariance Type:            nonrobust                                         
                                      coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
const     

In [20]:
X_TEST = sm.add_constant(X_test)
lr_pred = lr.predict(X_TEST)
scores = compute_scores(X_TEST, y_test, lr_pred)
save_scores('LinearRegression', scores)

MSE: 1566066389749368881152.0000
MAE: 30279372891.1281
MR: 7100327449.2158
adj-R2: -1563240973179226947584.0000


/Users/fra/miniconda3/envs/tensorflow/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/fra/miniconda3/envs/tensorflow/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/fra/miniconda3/envs/tensorflow/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/fra/miniconda3/envs/tensorflow/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `

In [21]:
export_pred(lr_pred, 'LinearRegression')

In [22]:
lr_pred

0       2.874459e+10
1       2.874459e+10
2       2.874459e+10
3       2.874459e+10
4       2.874459e+10
            ...     
2245    4.516497e-01
2246   -2.874459e+10
2247    5.695444e+10
2248    1.380003e-01
2249    1.112761e+00
Length: 2250, dtype: float64

# 5. Random Forest - only predict

In [23]:
yf= y.flatten()

In [24]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

In [25]:
best_forest = RandomForestRegressor(max_depth=30, n_estimators=128)
best_forest.fit(X, yf)

/Users/fra/miniconda3/envs/tensorflow/lib/python3.10/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/Users/fra/miniconda3/envs/tensorflow/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/fra/miniconda3/envs/tensorflow/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):


RandomForestRegressor(max_depth=30, n_estimators=128)

In [26]:
forest_pred = best_forest.predict(X_test)
scores = compute_scores(X_test, y_test, forest_pred)
save_scores('RandomForest', scores)

MSE: 0.1738
MAE: 0.3183
MR: -0.0061
adj-R2: 0.8266


/Users/fra/miniconda3/envs/tensorflow/lib/python3.10/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/Users/fra/miniconda3/envs/tensorflow/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/fra/miniconda3/envs/tensorflow/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):


In [27]:
len(forest_pred)

2250

In [28]:
export_pred(forest_pred, 'RandomForest')

# 6. XGBoost 

First we explore the default parameters, then move on to cross validation.

In [29]:
from xgboost import XGBRegressor
import time

In [30]:
best_params_xgb = {'colsample_bytree': 1.0, 'gamma': 0.2, 
                'learning_rate': 0.3, 'max_depth': 6, 
                'min_child_weight': 2, 'n_estimators': 128, 
                'reg_alpha': 0.5, 'reg_lambda': 1, 'subsample': 1.0}

In [31]:
# Creating a XGB model with the selected parameters
best_xgbr = XGBRegressor(max_depth=best_params_xgb['max_depth'],
                        n_estimators=best_params_xgb['n_estimators'],
                        learning_rate=best_params_xgb['learning_rate'],
                        min_child_weight=best_params_xgb['min_child_weight'], 
                        gamma=best_params_xgb['gamma'],
                        alpha=best_params_xgb['reg_alpha'],
                        reg_lambda=best_params_xgb['reg_lambda'],
                        subsample=best_params_xgb['subsample'],
                        colsample_bytree=best_params_xgb['colsample_bytree']
                        )
best_xgbr.fit(X, yf)

/Users/fra/miniconda3/envs/tensorflow/lib/python3.10/site-packages/xgboost/data.py:312: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
/Users/fra/miniconda3/envs/tensorflow/lib/python3.10/site-packages/xgboost/data.py:314: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  elif is_categorical_dtype(dtype) and enable_categorical:
/Users/fra/miniconda3/envs/tensorflow/lib/python3.10/site-packages/xgboost/data.py:345: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype)
/Users/fra/miniconda3/envs/tensorflow/lib/python3.10/site-packages/xgboost/data.py:336: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, Categorical

XGBRegressor(alpha=0.5, base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=1.0, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=0.2, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.3, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=6, max_leaves=None,
             min_child_weight=2, missing=nan, monotone_constraints=None,
             n_estimators=128, n_jobs=None, num_parallel_tree=None,
             predictor=None, ...)

In [32]:
xgb_pred = best_xgbr.predict(X_test)
scores = compute_scores(X_test, y_test, xgb_pred)

save_scores('XGBoost', scores)

MSE: 0.1689
MAE: 0.3158
MR: -0.0031
adj-R2: 0.8315


/Users/fra/miniconda3/envs/tensorflow/lib/python3.10/site-packages/xgboost/data.py:312: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
/Users/fra/miniconda3/envs/tensorflow/lib/python3.10/site-packages/xgboost/data.py:314: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  elif is_categorical_dtype(dtype) and enable_categorical:
/Users/fra/miniconda3/envs/tensorflow/lib/python3.10/site-packages/xgboost/data.py:345: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype)
/Users/fra/miniconda3/envs/tensorflow/lib/python3.10/site-packages/xgboost/data.py:336: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, Categorical

In [33]:
export_pred(xgb_pred, 'XGBoost')

# 7. Light GBM

In [34]:
from lightgbm import LGBMRegressor

Best LGBMRegressor with params: {'colsample_bytree': 1.0, 'learning_rate': 0.3, 'max_depth': 6, 'min_child_weight': 1, 'n_estimators': 128, 'num_leaves': 64, 'reg_alpha': 0.5, 'reg_lambda': 1} and MSE: -0.176


In [35]:
best_params_lgbm= {'colsample_bytree': 1.0, 'learning_rate': 0.3, 
              'max_depth': 6, 'min_child_weight': 1, 'n_estimators': 128, 
               'reg_alpha': 0.5, 'reg_lambda': 1, 'num_leaves': 64} 

In [36]:
# Creating a XGB model with the selected parameters
best_lgbm = LGBMRegressor(max_depth=best_params_lgbm['max_depth'],
                        n_estimators=best_params_lgbm['n_estimators'],
                        learning_rate=best_params_lgbm['learning_rate'],
                        min_child_weight=best_params_lgbm['min_child_weight'], 
                        reg_alpha=best_params_lgbm['reg_alpha'],
                        reg_lambda=best_params_lgbm['reg_lambda'],
                        colsample_bytree=best_params_lgbm['colsample_bytree'],
                        num_leaves=best_params_lgbm['num_leaves'],
                        )
best_lgbm.fit(X, yf)

LGBMRegressor(learning_rate=0.3, max_depth=6, min_child_weight=1,
              n_estimators=128, num_leaves=64, reg_alpha=0.5, reg_lambda=1)

In [37]:
lgbm_pred = best_lgbm.predict(X_test)
scores = compute_scores(X_test, y_test, lgbm_pred)

save_scores('LightGBM', scores)

MSE: 0.1725
MAE: 0.3201
MR: 0.0070
adj-R2: 0.8279


In [38]:
export_pred(lgbm_pred, 'LightGBM')

# 8. Hybrid Regression

In [39]:
# Import predictions from csv files, which have been rescaled to the original scale
forest_pred = pd.read_csv('predictions/RandomForest_pred.csv', header=None)
xgbr_pred = pd.read_csv('predictions/XGBoost_pred.csv', header=None)
lgbm_pred = pd.read_csv('predictions/LightGBM_pred.csv', header=None)

In [40]:
# Define the weights for each model
weights = [1/3, 1/3, 1/3]  # Equal weights for each model

# Combine predictions using weighted average
hybrid_predictions = (weights[0] * forest_pred + 
                      weights[1] * xgbr_pred +     
                      weights[2] * lgbm_pred)

# Standardize, scale to the same scale of y_test
hybrid_predictions = ss_y.transform(np.array(hybrid_predictions).reshape(-1, 1))

In [41]:
scores = compute_scores(X_test, y_test, hybrid_predictions)
save_scores('HybridModel', scores)

MSE: 0.1639
MAE: 0.3112
MR: -0.0007
adj-R2: 0.8364


In [42]:
export_pred(hybrid_predictions, 'HybridModel')

# 9. Stacking
Architecture from Quang

In [43]:
from vecstack import StackingTransformer

# Initialize 1st level estimators
estimators = [('RF', RandomForestRegressor(max_depth=30, #forest.best_params_['max_depth'], 
                                    n_estimators= 128 )),#forest.best_params_['n_estimators'])),
              ('LGBM', LGBMRegressor(max_depth=best_params_lgbm['max_depth'],
                        n_estimators=best_params_lgbm['n_estimators'],
                        learning_rate=best_params_lgbm['learning_rate'],
                        min_child_weight=best_params_lgbm['min_child_weight'], 
                        reg_alpha=best_params_lgbm['reg_alpha'],
                        reg_lambda=best_params_lgbm['reg_lambda'],
                        colsample_bytree=best_params_lgbm['colsample_bytree'],
                        num_leaves=best_params_lgbm['num_leaves'],
              ))]
              
# Initialize StackingTransformer
stack = StackingTransformer(estimators, regression=True, verbose=2, shuffle=True, metric=mean_squared_error)

In [44]:
stack = stack.fit(X, yf)

task:         [regression]
metric:       [mean_squared_error]
variant:      [A]
n_estimators: [2]

estimator  0: [RF: RandomForestRegressor]


/Users/fra/miniconda3/envs/tensorflow/lib/python3.10/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/Users/fra/miniconda3/envs/tensorflow/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/fra/miniconda3/envs/tensorflow/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):


    fold  0:  [0.18621050]
    fold  1:  [0.18430801]
    fold  2:  [0.18792116]
    fold  3:  [0.18771257]
    ----
    MEAN:     [0.18653806] + [0.00144680]

estimator  1: [LGBM: LGBMRegressor]
    fold  0:  [0.17604921]
    fold  1:  [0.17687877]
    fold  2:  [0.17925742]
    fold  3:  [0.17606261]
    ----
    MEAN:     [0.17706200] + [0.00131129]



In [45]:
# Get stacked features
S_train = stack.transform(X)
S_test = stack.transform(X_test)

Train set was detected.
Transforming...

estimator  0: [RF: RandomForestRegressor]


/Users/fra/miniconda3/envs/tensorflow/lib/python3.10/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/Users/fra/miniconda3/envs/tensorflow/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/fra/miniconda3/envs/tensorflow/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):


    model from fold  0: done
    model from fold  1: done
    model from fold  2: done
    model from fold  3: done
    ----
    DONE

estimator  1: [LGBM: LGBMRegressor]
    model from fold  0: done
    model from fold  1: done
    model from fold  2: done
    model from fold  3: done
    ----
    DONE

Transforming...

estimator  0: [RF: RandomForestRegressor]
    model from fold  0: done
    model from fold  1: done


/Users/fra/miniconda3/envs/tensorflow/lib/python3.10/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/Users/fra/miniconda3/envs/tensorflow/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/fra/miniconda3/envs/tensorflow/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):


    model from fold  2: done
    model from fold  3: done
    ----
    DONE

estimator  1: [LGBM: LGBMRegressor]
    model from fold  0: done
    model from fold  1: done
    model from fold  2: done
    model from fold  3: done
    ----
    DONE



In [46]:
xgbr = XGBRegressor()
start = time.time()
xgbr.fit(S_train, yf)
elapsed = time.time() - start
print(f'XGB Stacking Training ran in {elapsed:.5f} seconds')

XGB Stacking Training ran in 2.55562 seconds


In [47]:
stacking_pred = xgbr.predict(S_test)
scores = compute_scores(S_test, y_test, stacking_pred)

save_scores('Stacking', scores)

MSE: 0.1644
MAE: 0.3116
MR: -0.0050
adj-R2: 0.8411


In [48]:
export_pred(stacking_pred, 'Stacking')

# FINAL SUMMARY OF METHODS PERFORMANCES

We display below a summary of the scores obtained by the different models. 
We see that models can be classified differently based on the metric of evaluation chosen. 




In [49]:
final_scores=display_scores()

In [50]:
final_scores

,method,MSE,MAE,MR,adj R-squared
0,LinearRegression,1566066389749368881152.0,30279372891.128063,7100327449.215753,-1563240973179226947584.0
1,RandomForest,0.173842,0.318328,-0.006083,0.826551
2,XGBoost,0.168855,0.315781,-0.003086,0.831527
3,LightGBM,0.172455,0.320071,0.006979,0.827935
4,HybridModel,0.163925,0.311217,-0.00073,0.836446
5,Stacking,0.164427,0.311639,-0.004967,0.841129


In [51]:
final_scores.to_csv('final_scores.csv')

# analisi finali
- predictions per anno vs reale (grafica)